In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm
import multiprocessing
from functools import partial
from PIL import Image

In [ ]:
path = 'D:/HPCG/Adobe/labels/notsimulated_combine_male_short_outfits_genesis8_armani_casualoutfit03_Base_Pose_Standing_A'
output_folder = os.path.join(path, 'base')
gt_file = os.path.join(path, 'ground_truth.txt')
lines = []

with open(gt_file) as f:
    reader = csv.reader(f, delimiter=',')
    for r in reader:
        lines.append(r)

print('there are {} lines'.format(len(lines)))

In [ ]:
# construct the map
# [camera, human_rot] -> [[i, j]->[prefix]]
# oss << cur_prefix << ",";
# oss << light_pixel_pos.to_string() << ",";
# oss << to_string(manager.cur_camera->_position) << ",";
# oss << target_rot << ",";
# oss << to_string(render_target->compute_world_center()) << ",";
# oss << to_string(manager.m_lights[0]->m_verts[0]) << std::endl;
# gt_str.push_back(oss.str());
group_data = {}
for l in tqdm(lines):
    prefix = l[0]
    ibl = (int(l[1]), int(l[2]))
    camera_pos = (l[3], l[4], l[5])
    rot = l[6]
    target_center = (l[7], l[8], l[9])
    light_pos = (l[10], l[11], l[12])

    key = (camera_pos, rot)
    if key not in group_data.keys():
        group_data[key] = dict()

    ibl_key = ibl
    group_data[key][ibl_key] = prefix

In [ ]:
print('keys: ',len(group_data.keys()))
print('keys: ', group_data.keys())

# h x w x x x y 
h, w, x, y = 256,256, 512, 256-176 

print(h * w * x * y * 4 / 1024 / 1024 /1024)

all_data = np.zeros((h, w, x, y))

In [ ]:
info_str = ""
new_prefix = 0
img_folder = os.path.join(path, 'imgs')
x_begin, y_begin = 0, 176

for key in group_data.keys():
    # loading data
    for (ibl, prefix) in tqdm(group_data[key].items()):
        x, y = ibl
        if y < 176:
            continue
        
        if y > 255:
            print('y: {} is out of range, bug here'.format(y))
            break
        
        shadow_path = os.path.join(img_folder, prefix + '_shadow.png')
        shadow_np = plt.imread(shadow_path)
        
        cur_x, cur_y = x - x_begin, y - y_begin
        all_data[:,:, cur_x, cur_y] = shadow_np[:,:,0]
    
    break